In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

### Tokenization

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [0]:
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')],
    ['id','sentence'])

In [0]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',
                                pattern='\\W')

In [0]:
count_tokens = udf(lambda words:len(words),IntegerType())

In [0]:
tokenized = tokenizer.transform(sen_df)

In [0]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [0]:
tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [0]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [0]:
rg_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



### Stopwords removal

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sent_df = spark.createDataFrame([
    (0,['I','saw','the','green','horse']),
    (1,['Mary','had','a','little','lamb'])
    ],['Id','tokens'])

In [0]:
sent_df.show()

+---+--------------------+
| Id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [0]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [0]:
remover.transform(sent_df).show()

+---+--------------------+--------------------+
| Id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



### N-grams

In [0]:
from pyspark.ml.feature import NGram

In [0]:
words_df = spark.createDataFrame([
    (0, ['Hi','I','heard','about','Spark']),
    (1, ['I','wish','Java','could','use','case','classes']),
    (2, ['Logistic','regression','models','are','neat'])],
    ['Id','words'])

In [0]:
words_df.show(truncate=False)

+---+------------------------------------------+
|Id |words                                     |
+---+------------------------------------------+
|0  |[Hi, I, heard, about, Spark]              |
|1  |[I, wish, Java, could, use, case, classes]|
|2  |[Logistic, regression, models, are, neat] |
+---+------------------------------------------+



In [0]:
ngram = NGram(n=2,inputCol='words',outputCol='grams')

In [0]:
ngram.transform(words_df).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



### TF-IDF Vectorizer

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [0]:
sent_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'),
    (1, 'I wish Java could use case classes'),
    (2, 'Logistic Regression models are neat')],
    ['label','sentence'])

In [9]:
sent_df.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I heard about ...|
|    1|I wish Java could...|
|    2|Logistic Regressi...|
+-----+--------------------+



In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [0]:
words_data = tokenizer.transform(sent_df)

In [12]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|
|    1|I wish Java could...|[i, wish, java, c...|
|    2|Logistic Regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [0]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [0]:
featurized_data = hashing_tf.transform(words_data)

In [0]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [0]:
idf_model = idf.fit(featurized_data)

In [0]:
rescaled_data = idf_model.transform(featurized_data)

In [20]:
rescaled_data.select('label','features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|1    |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

### Count Vectorizer

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' '))],
    ['id','words'])

In [23]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [0]:
# a word appears in atleast 2 docs to appear in vocab
# minDF=2.0
cv = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3,minDF=2.0)

In [0]:
model = cv.fit(df)

In [0]:
result = model.transform(df)

In [27]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+



### Code Along - Spam Detection Filter

In [0]:
data = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')

In [0]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [31]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import length

In [0]:
data = data.withColumn('length',length(data['text']))

In [34]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [36]:
data.groupBy('class').mean().show() # length can be useful feature

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
cv = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,
                                  stop_remove,cv,idf,clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data)

In [0]:
clean_data = cleaner.transform(data)

In [49]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [0]:
clean_data = clean_data.select('label','features')

In [51]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [0]:
train,test = clean_data.randomSplit([0.7,0.3])

In [0]:
spam_detector = nb.fit(train)

In [0]:
test_results = spam_detector.transform(test)

In [55]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-610.85546689897...|[1.0,3.7829297063...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-848.43985695122...|[1.0,4.5683403142...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-690.21355905714...|[1.0,8.0598472485...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-673.41547015550...|[1.0,4.8212392214...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-831.53281889534...|[1.0,3.5555083574...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1527.3447817835...|[0.99999999472874...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-615.20663726364...|[0.99836287729809...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-251.11225743114...|[0.78763917898146...|       0.0|
|  0.0|(13424,[0,1,416,6...|[-301.56982978764...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0

In [0]:
# evaluate results
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [0]:
acc = acc_eval.evaluate(test_results)

In [60]:
print('Accuracy of NB model:')
print(acc)

Accuracy of NB model:
0.9122807017543859
